In [ ]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model 

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al azar''.
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

from mesa.space import ContinuousSpace

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime#

In [ ]:
class FlockAgent(Agent):
    def __init__(self, id, model, x, y):
        super().__init__(id, model)

        self.position = np.array((x, y), dtype=np.float64)
        
        vec = (np.rand(2) - 0.5) * 10
        self.velocity = np.array(vec, dtype=np.float64)
        
        vec = (np.rand(2) - 0.5) / 2
        self.acceleration = np.array(vec, dtype=np.float64)

        self.max_force = 0.3
        self.max_speed = 5
        self.perception = 50

    def step(self):
        self.check_edges()
        self.check_with_neighbours()
        
        self.position = self.position + self.velocity
        self.velocity = self.velocity + self.acceleration

        if np.linalg.norm(self.velocity) > self.max_speed:
            self.velocity = (self.velocity / np.linalg.norm(self.velocity)) * self.max_speed
        
        self.acceleration = np.array((0, 0), dtype=np.float64)
        self.model.grid.move_agent(self, self.position)

    def check_edges(self):
        (x, y) = self.position

        if x > self.model.grid.x_max:
            x = self.model.grid.x_min
        elif x < self.model.grid.x_min:
            x = self.model.grid.x_max

        if y > self.model.grid.y_max:
            y = self.model.grid.y_min
        elif y < self.model.grid.y_min:
            y = self.model.grid.y_max

        self.position = np.array((x, y), dtype=np.float64)
    
    def check_with_neighbours(self):
        alignment = self.align()
        cohesion = self.cohesion()
        separation = self.separation()

        self.acceleration = self.acceleration + alignment + cohesion + separation